In this notebook, I use a fictional dataset to demonstrate a very common data manipulation problem I run into in Audit Analytics projects: given the weight factors across many groups, how to calculate the percentage weight for each members within a group. For example, in the following dataset, the column "Contribution_Factor" is used to allocate the department total Bonus: the total bonus for a department will be allocated to the employees in this department proportional to the employee's contribution factor. 

# 1. Import the two datasets ("Salary" and "Bonus" ) from Excel Sheets

In [11]:
import pandas as pd

In [12]:
salary=pd.read_excel("Data.xlsx",sheet_name="Salary")
salary.head()

,Employee_ID,Department,Base_Salary,Contribution_Factor
0,E301,Finance,69612,50
1,E313,Finance,67716,101
2,E224,Finance,36132,59
3,E186,Finance,64092,103
4,E38,Finance,55428,67


In [33]:
bonus=pd.read_excel("Data.xlsx",sheet_name="Bonus")
bonus.head()

,Department,Bonus_Sum
0,Finance,300000
1,Management,510000
2,Administration,100000
3,Production,453000
4,After_Sales Service,506600


# 2. Merge the two dataset on the common column "department"

In [14]:
salary_bonus=pd.merge(salary,bonus,how="inner",on="Department")
salary_bonus.head()

,Employee_ID,Department,Base_Salary,Contribution_Factor,Bonus_Sum
0,E301,Finance,69612,50,300000
1,E313,Finance,67716,101,300000
2,E224,Finance,36132,59,300000
3,E186,Finance,64092,103,300000
4,E38,Finance,55428,67,300000


# 3. Calculate the Percentage weight for each employee within each group and allocate the department wide bonus to employees according to the Percentage weight.

In calculating the percentage weight of each employee within a group, I first use groupby with the first level of indices of the dataframe to split the dataset into 6 groups (departments) and then apply the transformation function "sum" on each members of the group. It is important to note that the function transform return an output with the same shape as the input. This is the key for the allocation method. 

In [15]:
salary_bonus.set_index('Department',inplace=True)

salary_bonus.loc[:,"share"]=salary_bonus.loc[:,"Contribution_Factor"]/salary_bonus.groupby(level=0)["Contribution_Factor"].transform(sum)

salary_bonus.loc[:,"salary_with_bonus"]=salary_bonus.loc[:,"Base_Salary"]+salary_bonus.loc[:,"Bonus_Sum"]*salary_bonus.loc[:,"share"]

salary_bonus.head(5)

,Employee_ID,Base_Salary,Contribution_Factor,Bonus_Sum,share,salary_with_bonus
Department,,,,,,
Finance,E301,69612,50,300000,0.013543,73674.838570
Finance,E313,67716,101,300000,0.027356,75922.933911
Finance,E224,36132,59,300000,0.015980,40926.149512
Finance,E186,64092,103,300000,0.027898,72461.447454
Finance,E38,55428,67,300000,0.018147,60872.203684


# 4. Test the allocation result

### 4.1 check whether the sum of percentages within a group is equal to 1.

In [16]:
salary_bonus.groupby(level=0)["share"].sum()

Department
Administration         1.0
After_Sales Service    1.0
Finance                1.0
Management             1.0
Marketing              1.0
Production             1.0
Name: share, dtype: float64

### 4.2 check wether department bonus is allocated to each employee within that department according to employee's contribution factor.

In [34]:
bonus.set_index("Department",inplace=True)
bonus

,Bonus_Sum
Department,
Finance,300000
Management,510000
Administration,100000
Production,453000
After_Sales Service,506600
Marketing,300000


In [35]:
bonus_re=(salary_bonus["salary_with_bonus"]-salary_bonus["Base_Salary"])/salary_bonus["share"]
bonus_re=bonus_re.to_frame(name="Bonus_Re")
bonus_re


,Bonus_Re
Department,
Finance,300000.0
Finance,300000.0
Finance,300000.0
Finance,300000.0
Finance,300000.0
...,...
Production,453000.0
Production,453000.0
Production,453000.0


Join the two dataframes on index and compare the two columns in order to test allocation.

In [37]:
test=bonus_re.join(bonus,how="inner")
test.loc[:,"Comparison"]=test.loc[:,"Bonus_Re"]-test.loc[:,"Bonus_Sum"]
test

,Bonus_Re,Bonus_Sum,Comparison
Department,,,
Administration,100000.0,100000,-5.820766e-11
Administration,100000.0,100000,0.000000e+00
Administration,100000.0,100000,-2.910383e-11
Administration,100000.0,100000,-1.018634e-10
Administration,100000.0,100000,-2.182787e-10
...,...,...,...
Production,453000.0,453000,1.746230e-10
Production,453000.0,453000,-5.820766e-10
Production,453000.0,453000,9.313226e-10


The allocation is almost perfect. There is some very small allocation error remaining.

In [38]:
test.Comparison.sum()

-6.679329089820385e-09